# 03 – Estimation (MLE with Monte-Carlo marginalisation & Stan skeleton)

In this notebook, we fit the mixed hidden Markov model to a simulated dataset
(`data/simulated/ref_scenario.csv`) generated in `02_simulator.ipynb`.

It contains:
- A Monte-Carlo marginalization approach for MLE (approximate marginal likelihood)
- A CmdStanPy / Stan model skeleton for Bayesian inference (marginalize discrete states via forward algorithm)

**Notes:**
- MLE via MC is computationally heavy (many forward calls); we provide a small quick test and an option to scale up.
- The Stan skeleton is illustrative and requires more tuning & priors to run for production.


In [1]:
import sys, os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")
utils_path = os.path.join(project_root, "utils")

for p in [project_root, src_path, utils_path]:
    if p not in sys.path:
        sys.path.insert(0, p)

print("Project root:", project_root)
print("src path:", src_path)
print("utils path:", utils_path)


Project root: /Users/user/Documents/Projects/Markov-Models/mHMM
src path: /Users/user/Documents/Projects/Markov-Models/mHMM/src
utils path: /Users/user/Documents/Projects/Markov-Models/mHMM/utils


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from tqdm.notebook import tqdm

import time
import pickle
import os

from src.emissions import EmissionModel
from src.transitions import TransitionModel

Stable log-sum-exp and forward using EmissionModel.logpdf  (check eqtn 11 in the paper):

In [3]:
EPS = 1e-12

def logsumexp_arr(a):
    m = np.max(a)
    return m + np.log(np.sum(np.exp(a-m)))

def forward_loglik_subject(obs, times, init_probs, trans_mat, emission_model, g):

    """Compute the log-likelihood of a single subject given fixed
    individual effects g using the forward algorithm in log-space for numerical stability.
    Args:
        obs (np.ndarray): Observations for the subject (T x D).
        times (np.ndarray): Observation times for the subject (T,).
        g: dict of random effects for the subject.
        """
    T = len(obs)
    n_states = 2
    #initialize log alpha
    log_alpha = np.zeros((T, n_states)) # log forward probabilities

    for s in range(n_states):
        log_em = emission_model.logpdf(obs[0], g, times[0], s)
        log_alpha[0, s] = np.log(init_probs[s] + EPS) + log_em 

    #normalize in log-space by subtracting logsumexp
    for t in range(1, T): #forward recursion
        new_log_alpha = np.full(n_states, -np.inf)
        for j in range(n_states):
            #compute logsum over previous states
            prev = log_alpha[t-1, :] + np.log(trans_mat[:, j] + EPS) 
            s_prev = logsumexp_arr(prev)
            log_em = emission_model.logpdf(obs[t], g, times[t], j)
            new_log_alpha[j] = s_prev + log_em #combine transition and emission log-probs
        log_alpha[t, :] = new_log_alpha 
    
    return logsumexp_arr(log_alpha[-1, :]) #marginal log-likelihood is logsumexp of final alphas  

Load simulated data and prep per-subject datasets

In [4]:
data_path = '../data/simulated/ref_scenario.csv'
assert os.path.exists(data_path), f"Data file not found at {data_path}. Run 02_simulator.ipynb to generate it."

df = pd.read_csv(data_path)
df.head()

,ID,Week,State,FEV1,PRO
0,1,0,0,1.651523,2.803810
1,1,1,1,1.678107,3.123523
2,1,2,1,1.348596,3.448288
3,1,3,0,1.620862,2.832255
4,1,4,1,1.501967,3.267526


In [5]:
subjects = df['ID'].unique()
subj_data = {}

for sid in subjects:
    sub = df[df.ID ==sid].sort_values("Week")
    obs = sub[["FEV1","PRO"]].values
    times = sub["Week"].values.astype(float)
    subj_data[int(sid)] = {'obs': obs, 'times': times}

len(subj_data), list(subj_data.keys())[:5] # Check number of subjects and first 5 IDs

(100, [1, 2, 3, 4, 5])

Monte_Carlo marginal log-likelihood per subject

-For each candidate population parameter set, for each subject we:

-> draw K samples of g ~ N(0, x2_*) using the candidate x2 variances,

-> compute forward log-likelihood for each g,

-> average the likelihoods (in log-space using log-sum-exp) to approximate the marginal likelihood.

Since this is slow for many subjects / large K, it's advisable to use small tests (e.g., K=30) before scaling.

In [6]:
def sample_g_matrix_em_params(n_samples, rng, x2_params):
    """
    Draw n_samples of g according to x2 variances dict.
    x2_params: dict with keys ['x2_FEV1R','x2_FEV1E','x2_PROR','x2_PROE']
    Returns list of dicts length n_samples.
    """
    s_FEV1R = np.sqrt(x2_params['x2_FEV1R'])  #standard deviations
    s_FEV1E = np.sqrt(x2_params['x2_FEV1E'])
    s_PROR = np.sqrt(x2_params['x2_PROR'])
    s_PROE = np.sqrt(x2_params['x2_PROE'])
    rng = np.random.default_rng(rng)
    gs = []
    for _ in range(n_samples): #draw n_samples
        g = {
            "gFEV1R": float(rng.normal(0.0, s_FEV1R)),
            "gFEV1E": float(rng.normal(0.0, s_FEV1E)),
            "gPROR":  float(rng.normal(0.0, s_PROR)),
            "gPROE":  float(rng.normal(0.0, s_PROE))
        }
        gs.append(g)
    return gs


In [7]:
import importlib
import utils.sampling_utils as su
importlib.reload(su) 

<module 'utils.sampling_utils' from '/Users/user/Documents/Projects/Markov-Models/mHMM/utils/sampling_utils.py'>

In [8]:
def approx_subject_log_marginal(obs, times, init_probs, trans_mat, em_params, K=30, rng_seed=None):
    """Approximate the marginal log-likelihood of a subject
    by Monte-Carlo integration over g.
    Args:
        obs (np.ndarray): Observations for the subject (T x D).
        times (np.ndarray): Observation times for the subject (T,).
        em_params: dict of emission parameters including variance components.
        K (int): Number of Monte-Carlo samples."""

    #build emission model from em_params
    em = EmissionModel(
        hFEV1R=em_params['hFEV1R'],
        hFEV1E=em_params['hFEV1E'],
        x2_FEV1R=em_params['x2_FEV1R'],
        x2_FEV1E=em_params['x2_FEV1E'],
        hPROR=em_params['hPROR'],
        hPROE=em_params['hPROE'],
        x2_PROR=em_params['x2_PROR'],
        x2_PROE=em_params['x2_PROE'],
        r2_FEV1=em_params['r2_FEV1'],
        r2_PRO=em_params['r2_PRO'],
        qR=em_params['qR'],
        qE=em_params['qE'],
        PE=em_params['PE'],
        PHL=em_params['PHL']
    )

    #sample K g's
    x2_params = {k: em_params[k] for k in ['x2_FEV1R','x2_FEV1E','x2_PROR','x2_PROE']} 
    gs = sample_g_matrix_em_params(K, rng_seed, x2_params)
    #compute log-likelihoods for each g
    logls = []
    for g in gs:
        logl_g = forward_loglik_subject(obs, times, init_probs, trans_mat, em, g)
        logls.append(logl_g)
    #combine via log-mean-exp: log(1/k * sum exp(logl_g ))
    logls = np.array(logls)
    lm = logsumexp_arr(logls) - np.log(K)
    return lm

## Full data negative log-likelihood to optimize

We assemble a function neg_loglik(params) that maps a parameter vector to the negative log-likelihood across subjects using the Monte Carlo approximation.
For clarity we will optimize a reduced parameter set in this notebook

Examples of parameters we estimate here:

-> hFEV1R, hFEV1E, hPROR, hPROE (modes)

-> log of IIV variances: log_x2_FEV1R, log_x2_FEV1E, log_x2_PROR, log_x2_PROE

-> log of residual variances: log_r2_FEV1, log_r2_PRO

-> Fisher-transformed correlations btn the variables: atanh(qR), atanh(qE)

-> hpRE and hpER (we optimize them on logit scale)

For a quick test we only fit a subset (e.g., hFEV1R, hPROR, log residuals). The fixed_mask is for fixing parameters if desired.

In [9]:
def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

def logit(p):
    p = np.clip(p, EPS, 1.0 - EPS)
    return np.log(p / (1.0 - p))
# Parameter indexing helpers (order)
param_names = [
    'hFEV1R','hFEV1E','hPROR','hPROE',
    'log_x2_FEV1R','log_x2_FEV1E','log_x2_PROR','log_x2_PROE',
    'log_r2_FEV1','log_r2_PRO',
    'atanh_qR','atanh_qE',
    'logit_hpRE','logit_hpER'
]

def pack_params_from_vector(vec):
    """
    Convert unconstrained vector to em_params dict and transition params.
    `vec` must follow the param_names order.
    """
    d = {}
    idx = 0
    d['hFEV1R'] = vec[idx]; idx+=1  #must be positive  
    d['hFEV1E'] = vec[idx]; idx+=1
    d['hPROR']  = vec[idx]; idx+=1
    d['hPROE']  = vec[idx]; idx+=1

    d['x2_FEV1R'] = np.exp(vec[idx]); idx+=1
    d['x2_FEV1E'] = np.exp(vec[idx]); idx+=1
    d['x2_PROR']  = np.exp(vec[idx]); idx+=1
    d['x2_PROE']  = np.exp(vec[idx]); idx+=1

    d['r2_FEV1'] = np.exp(vec[idx]); idx+=1
    d['r2_PRO']  = np.exp(vec[idx]); idx+=1

    d['qR'] = np.tanh(vec[idx]); idx+=1
    d['qE'] = np.tanh(vec[idx]); idx+=1

    d['hpRE'] = logistic(vec[idx]); idx+=1
    d['hpER'] = logistic(vec[idx]); idx+=1

    # placeholders for PE, PHL (fixed)
    d['PE'] = 0.2
    d['PHL'] = 10.0
    return d

# default initial vector set to values close to simulation
init_vec = np.array([
    3.0, 0.5,  # hFEV1R, hFEV1E
    2.5, 0.5,  # hPROR, hPROE
    np.log(0.03), np.log(0.03), np.log(0.09), np.log(0.09),  # log x2
    np.log(0.015), np.log(0.05),  # log r2
    np.arctanh(-0.33), np.arctanh(-0.33),  # atanh q
    logit(0.1), logit(0.3)  # logit hpRE, hpER
])

# Choose a subset to optimize for a quick demo (e.g., first 10 params)
free_idx = list(range(len(init_vec)))  # by default free all
# for a faster demo you might limit:
# free_idx = list(range(10))  # fit only the first 10 parameters

def neg_loglik_mc(vec, subj_data_local, init_probs, trans_mat_fn, K=30, rng_seed=42):
    """
    Negative log-likelihood across all subjects approximated by MC integration.
    trans_mat_fn: function that given em_params returns transition matrix (we keep hpRE/hpER in params)
    """
    em_params = pack_params_from_vector(vec)
    # build trans_mat from hpRE/hpER (no gpRE/gpER here)
    tm = TransitionModel(hpRE=em_params['hpRE'],gpRE=0.0, hpER=em_params['hpER'], gpER=0.0) 
    trans_mat = tm.transition_matrix()

    total_logl = 0.0
    rng_seq = np.random.SeedSequence(rng_seed)
    # Loop subjects (consider batching / parallelization later)
    for i, sid in enumerate(subj_data_local.keys()):
        d = subj_data_local[sid]
        lm = approx_subject_log_marginal(
            d['obs'], d['times'],
            init_probs, trans_mat, em_params, K=K, rng_seed=rng_seq.spawn(1)[0].entropy
        )
        total_logl += lm
    return -float(total_logl)


Quick test of the fn neg_loglik with a small subset 

In [10]:
test_subjects = {k: subj_data[k] for k in list(subj_data.keys())[:5]}  # first 5 subjects for quick test
init_probs = np.array([0.9, 0.1])  

t0 = time.time()
nl = neg_loglik_mc(init_vec, test_subjects, init_probs, None, K=20, rng_seed=1)
t1 = time.time()
print(f"Neg log-likelihood (5 subjects, K=20): {nl:.3f} computed in {t1 - t0:.1f} seconds")


Neg log-likelihood (5 subjects, K=20): 464.740 computed in 0.5 seconds


# Optimization demo

-> We minimize the negative log-likelihood of our mHMM model to find the params that maximizie the likelihood of our observed data.

-> We use L-BFGS-B algorithm - gradient-based optimization algorithm that efficiently handles large numbers of parameters and parameter bounds (e.g. variances > 0). More stable than basic gradient-descent.

In [11]:
subject_subset = {k: subj_data[k] for k in list(subj_data.keys())[:10]}  

#show progress 
history = []
def cb(xk):
    history.append(xk.copy())
    print('.', end='') 

res = minimize(
    fun = lambda v: neg_loglik_mc(v, subject_subset, init_probs, None, K=30, rng_seed=1),
    x0=init_vec,
    method='L-BFGS-B',
    options={'disp': True, 'maxiter': 50}
)

print("\nOptimization result:")
print(res) 


Optimization result:
  message: STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT
  success: False
   status: 1
      fun: 20.98552719714403
        x: [ 1.816e+00  2.048e-01 ... -3.983e+00 -1.158e+00]
      nit: 50
      jac: [-3.146e+01  1.319e+01 ...  6.740e-01  3.392e-01]
     nfev: 975
     njev: 65
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>


### Intepretation:
> Our L-BFGS-B hit the max iter limit (50) and stopped. The "false" is bc the optimizer didn't find a stopping condition.
> The "fun" is the negative log-likelihood value at the final param vector, which is in a normal range for the 10 subjects
> x is our em_params list
> nfev represents 975 negative-log likelihood evaluations and "nit" is for 50 optimizer steps
> We will increase the optimization quality later to 200 for maxiter and k=100. This was for testing our pipeline only.

# Interpret & decode Viterbi using a point estimate

In [12]:
if res.success:
    fitted_params = pack_params_from_vector(res.x)
    tm_hat = TransitionModel(hpRE=fitted_params['hpRE'], hpER=fitted_params['hpER'])
    trans_mat_hat = tm_hat.transition_matrix()
    em_hat = EmissionModel(
        hFEV1R=fitted_params['hFEV1R'], hFEV1E=fitted_params['hFEV1E'],
        x2_FEV1R=fitted_params['x2_FEV1R'], x2_FEV1E=fitted_params['x2_FEV1E'],
        hPROR=fitted_params['hPROR'], hPROE=fitted_params['hPROE'],
        x2_PROR=fitted_params['x2_PROR'], x2_PROE=fitted_params['x2_PROE'],
        r2_FEV1=fitted_params['r2_FEV1'], r2_PRO=fitted_params['r2_PRO'],
        qR=fitted_params['qR'], qE=fitted_params['qE'],
        PE=fitted_params['PE'], PHL=fitted_params['PHL']
    )

    from mHMM.src.mhmm_forward import viterbi  
    # Naive g=0
    g0 = {"gFEV1R":0.0,"gFEV1E":0.0,"gPROR":0.0,"gPROE":0.0}
    sample_sid = list(subject_subset.keys())[0]
    sdata = subject_subset[sample_sid]
    states_decoded = viterbi(sdata['obs'], np.array([0.9,0.1]), trans_mat_hat, em_hat, g0, sdata['times'])
    print("Decoded states (naive g=0):", states_decoded)


Run CmdStanPy Sampling

## Summary 

In this notebook, we did deterministic Maximum Likelihood Estimation (MLE) for the mixed Hidden Markov Model (mHMM) using Python, without Stan. The main steps were:

1. Data Preparation

Loaded the simulated reference dataset from 02_simulator.ipynb and reshaped it into the format required for likelihood evaluation:

Flattened the longitudinal outcomes (FEV1, PRO)

Generated time, subject start indices, and lengths

Constructed a clean stan_data-like structure for use in Python

This ensures each subject’s complete time series can be passed to the likelihood function.

2. Monte Carlo Marginal Likelihood

Implemented the subject-specific marginal likelihood under random effects by:

Sampling individual random effects > 𝑔 using the population variance parameters

Evaluating state‐conditional emission probabilities

Propagating forward probabilities through the transition model

Averaging over Monte Carlo draws to approximate the integral

Aggregating contributions across subjects

This forms the full log-likelihood for the mHMM.

3. Maximum Likelihood Estimation (L-BFGS-B)

Set up an unconstrained parameter vector, applied transforms (logit, log, atanh) for valid parameter domains, and optimized the negative log-likelihood using:

L-BFGS-B

A limited number of iterations (for testing pipeline correctness)

While the test run stops at the iteration limit, it confirms that:

The likelihood is well-defined

Gradients can be computed

Optimization moves toward a plausible region

4. Recovering Model Parameters

After optimization, we unpacked the fitted parameter vector into:

Emission parameters

Random-effect variances

Transition parameters

Placebo/time effect parameters

This produces a complete set of point estimates for all population-level quantities in the mHMM.

5. Viterbi Decoding (Optional Demonstration)

Using the estimated parameters, we decoded the most likely latent state sequence for one sample subject using the Viterbi algorithm.
This provides an interpretable check of how the estimated model classifies latent responder / exacerbation states.

## Overall Purpose of This Notebook

This notebook implements the frequentist MLE pipeline for the mixed HMM:

Deterministic

Fast to test

Useful for debugging parameterizations and data flow

Matches the "MC-MLE" approach used as a baseline before SSE/SAEM
